In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

# set dataframe outputs to three digits 
pd.set_option("display.precision", 3)
#suppress warnings
warnings.filterwarnings('ignore')

## Functions

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df_k = pd.read_csv(path)
df_k.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

In [3]:
def r_square(model):
    # McKelvay-Zaviona
    xb = model.predict(linear=True) #fitted latent value
    var_xb = np.var(xb,ddof=1) # variance of xb
    r2_mz = var_xb / (var_xb + 1) # McKelvay-Zavoina R_2
    # McFaden
    r2_mf = model.prsquared
    return r2_mz

In [4]:
def probit(functions, data):
    results_list = []
    for func in functions:
        model = smf.probit(formula=func, data=data).fit(disp=0)
        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,    
            'pseudoR_2': r_square(model),
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_marginal = model.get_margeff().summary_frame()
        df_model = pd.concat([df_model, df_marginal], axis =1)

        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [5]:
#duplicate but with logit
def logit(functions, data):
    results_list = []
    for func in functions:
        model = smf.logit(formula=func, data=data).fit(disp=0)
        marg_effects = model.get_margeff().summary_frame()

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'marginal_effect': marg_effects['dy/dx'],
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

## What is the combined effect of experience, awareness, and flood zone on preapredness?

In [6]:
functions = [
    'made_safer ~ experience + awareness + floodzone',
    'documents ~ experience + awareness + floodzone',
    'insured ~ experience + awareness + floodzone',
    'learned_routes ~ experience + awareness + floodzone',
    'supplies ~ experience + awareness + floodzone',
    'involved ~ experience + awareness + floodzone',
    'made_plan ~ experience + awareness + floodzone',
    'practiced_drills ~ experience + awareness + floodzone',
    'alerts ~ experience + awareness + floodzone',
    'family_communication ~ experience + awareness + floodzone'
]

In [7]:
results = probit(functions=functions, data=df_k)
results = results.round(3) # set to three decimal places 
results.to_excel('results/Probit_DeterminantsOfRiskPerception.xlsx')

In [8]:
results = logit(functions=functions, data=df_k)
results = results.round(3) # set to three decimal places 
results.to_excel('results/Logit_DeterminantsOfRiskPerception.xlsx')

## Checking for multicollinearity 
&
## Prediction risk perception 

In [ ]:
functions = [
    'awareness ~ experience + floodzone',
    'experience ~ awareness + floodzone',
    'floodzone ~ awareness + experience',
    'perception ~ awareness + experience + floodzone'
]

In [10]:
results = probit(functions=functions, data=df_k)
results = results.round(3) # set to three decimal places 
results.to_excel('results/probit_robustnesscheck.xlsx')

In [11]:
results = logit(functions=functions, data=df_k)
reults = results.round(3)
results.to_excel('results/logit_robustnesscheck.xlsx')

## Check if perception predicts awareness?

In [ ]:
probit(functions = ['awareness ~ perception'], data=df_k).round(3)

effect    p  pseudoR_2  LLPr      BIC  \
function               beta                                                
awareness ~ perception Intercept   -0.447  0.0      0.111   0.0  515.788   
                       perception   0.710  0.0      0.111   0.0  515.788   

                                   dy/dx  Std. Err.      z  Pr(>|z|)  \
function               beta                                            
awareness ~ perception Intercept     NaN        NaN    NaN       NaN   
                       perception  0.266      0.043  6.121       0.0   

                                   Conf. Int. Low  Cont. Int. Hi.  
function               beta                                        
awareness ~ perception Intercept              NaN             NaN  
                       perception           0.181           0.351

In [20]:
logit(functions = ['awareness ~ perception'], data=df_k).round(3)

effect    p  marginal_effect  pseudoR_2  \
function               beta                                                  
awareness ~ perception Intercept   -0.720  0.0              NaN      0.055   
                       perception   1.141  0.0            0.263      0.055   

                                   LLPr      BIC  
function               beta                       
awareness ~ perception Intercept    0.0  515.788  
                       perception   0.0  515.788